<a href="https://colab.research.google.com/github/xwu391/6258-DIP/blob/master/ResNet_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dippykit

In [ ]:
import random
import numpy as np
import torch
import torchvision
from  torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from  torchvision import datasets, transforms
import dippykit as dip
from torchsummary import summary
import os
import skimage

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive


In [ ]:
def Make_Layer(inchannel, outchannel, Unit_Num):
  shortcut = nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=1, stride=1)
  Block_Layer = []
  for i in range(Unit_Num):
    Block_Layer.append(Residual_Block_Unit(64, 64, None))
  return nn.Sequential(*Block_Layer)

def Res_Layers(inchannel, outchannel, Block_Num):
  shortcut = nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=1, stride=1)
  Res_Layer = []
  for i in range(Block_Num):
    Res_Layer.append(Residual_Block(64, 64, None))
  Res_Layer.append(Residual_Block_Unit(64, 64, None))
  return nn.Sequential(*Res_Layer)

def Heat_Map_Maker(inchannel, outchannel, Unit_Num):
  shortcut = nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=1, stride=1)
  Map_Layer = []
  for i in range(Unit_Num):
    Map_Layer.append(Residual_Block_Unit(4, 4, None))
  return nn.Sequential(*Map_Layer)

In [ ]:
class Residual_Block_Unit(nn.Module):
  def __init__(self, inchannel, outchannel, shortcut=None):
    super(Residual_Block_Unit, self).__init__()
    self.Unit_Left = nn.Sequential(
        nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=3, stride=1, padding=1), 
        nn.BatchNorm2d(outchannel),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=3, stride=1, padding=1), 
        nn.BatchNorm2d(outchannel),
        nn.ReLU(inplace=True)
    )

    self.Unit_Right = shortcut

  def forward(self, input_im):
    out = self.Unit_Left(input_im)
    bypass = input_im if self.Unit_Right is None else self.Unit_Right(x)
    out = out + bypass
    return F.relu(out)

In [ ]:
class Residual_Block(nn.Module):
  def __init__(self, inchannel, outchannel, shortcut=None):
    super(Residual_Block, self).__init__()
    self.Block_Left = Make_Layer(64, 64, 3)
    self.Block_Right = shortcut

  def forward(self, input_im):
    out = self.Block_Left(input_im)
    bypass = input_im if self.Block_Right is None else self.Block_Right(x)
    out = out + bypass
    return F.relu(out)

In [ ]:
class ResNet_AE(nn.Module):
  def __init__(self):
    super(ResNet_AE, self).__init__()
    self.Pre = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5, stride=2, padding=2),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=2, padding=2),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True)
    )

    self.Res_Part_En = Res_Layers(64, 64, 2)
    self.Res_Part_Shortcut_En = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1, stride=1)
    self.Res_Encoder_Output = nn.Conv2d(in_channels=64, out_channels=4, kernel_size=5, stride=2, padding=2)

    self.Heat_Map_Layer = Heat_Map_Maker(4, 4, 1)

    self.Res_Decoder_Input = nn.ConvTranspose2d(in_channels=4, out_channels=64, kernel_size=4, stride=2, padding=1)
    self.Res_Part_De = Res_Layers(64, 64, 2)
    self.Res_Part_Shortcut_De = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1, stride=1)

    self.After = nn.Sequential(
        nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(in_channels=64, out_channels=1, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(1)
    ) 

  def Encoder(self, input_im):
    x_in = self.Pre(input_im)
    x_Left_En = self.Res_Part_En(x_in)
    x_Right_En = x_in if self.Res_Part_Shortcut_En is None else self.Res_Part_Shortcut_En(x_in)
    x = x_Left_En + x_Right_En
    x = F.relu(x)
    x = self.Res_Encoder_Output(x)
    x = F.sigmoid(x)
    return x

  def Importance_Map(self, input_im):
    Importance_Extracted = self.Heat_Map_Layer(input_im)
    Importance_Extracted = F.sigmoid(Importance_Extracted)
    with torch.no_grad():
   #   Expo_Map = torch.ones(Importance_Extracted.shape)
   #   Expo_Map = torch.mul(Expo_Map, 4)
      Quant_im = self.Hard_Quantizer(input_im, 6)
      Imp_Map = self.Hard_Quantizer(Importance_Extracted, 3)
      Imp_Map = torch.sub(Imp_Map, 3)
      Expos = torch.pow(2, Imp_Map)
      Code_Map = torch.mul(Expos, Quant_im)
      Code_Map = torch.div(torch.floor(Code_Map), 64)
    return Code_Map

  def Hard_Quantizer(self, input_im, bits_Num):
    out = torch.mul(input_im, 2**bits_Num)
    out = torch.floor(out)
    return out

  def Decoder(self, input_im):
    x = self.Res_Decoder_Input(input_im)
    x = F.relu(x)
    x_Left_De = self.Res_Part_De(x)
    x_Right_De = x if self.Res_Part_Shortcut_De is None else self.Res_Part_Shortcut_De(x)
    x = x_Left_De + x_Right_De
    x = F.relu(x)
    x = self.After(x) 
  #  x = self.FSRCNN(x)
  #  with torch.no_grad():
  #    mean = [0.434, 0.443, 0.367]
  #    std = [0.238, 0.223, 0.232]
  #    t_mean = torch.FloatTensor(mean).view(3,1,1).expand(3, 32, 32).cuda()
  #    t_std = torch.FloatTensor(std).view(3,1,1).expand(3, 32, 32).cuda()
  #    x_out = torch.mul(x, t_std)
  #    x_out = torch.add(x_out, t_mean)
    return F.sigmoid(x)

  def forward(self, input_im):
    Code = self.Encoder(input_im)
    Code_Map = self.Importance_Map(Code)
    recover = self.Decoder(Code)
    return recover, Code


In [ ]:
Pre_transfrom = transforms.Compose([
        transforms.Resize((128,128)),
        transforms.ToTensor(),
    ])

train_dataset = datasets.MNIST(root='./data/', train=True, transform=Pre_transfrom, download=True)
test_dataset = datasets.MNIST(root='./data/', train=False, transform=Pre_transfrom)

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=100, num_workers=10, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=4, num_workers=4, drop_last=True)

Res_AE = ResNet_AE().cuda()
optimizer = optim.SGD(Res_AE.parameters(), lr=0.001, momentum=0.9)

loss_function = nn.MSELoss()

for epoch in range(2):
  print("Epoch %d" % epoch)

  for i, (images, _) in enumerate(train_loader):
    optimizer.zero_grad()
    images = Variable(images.data, requires_grad = False).cuda()
    recover, code_map = Res_AE(images)
    zero_comp = torch.zeros(code_map.shape)
  #  recover = Variable(recover.data, requires_grad = True).cuda()
    loss = loss_function(recover, images)
    
    loss.backward()
    optimizer.step()        
        
  print("Loss = %.5f" % loss.item()) 
  if (epoch%20==0):
    torch.save(Res_AE, 'Res_AE_48.pkl')

print('Finished Training')


Epoch 0
Loss = 0.05607
Epoch 1


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

Res_AE = torch.load('Res_AE_48.pkl')

Pre_transfrom = transforms.Compose([
        transforms.Resize((128,128)),
        transforms.Grayscale(),
        transforms.ToTensor(),
    ])

test_dataset = datasets.ImageFolder(root='test/', transform=Pre_transfrom)

test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=1, num_workers=1, drop_last=True)

for i, (images, _) in enumerate(test_loader):
    images = Variable(images).cuda()
    recover, code = Res_AE(images)
    torchvision.utils.save_image(recover.data, 'Res_Rec49' + str(i) + '.png')

In [ ]:
import dippykit as dip
print(code.shape)
code1 = torch.mul(code, 64).reshape(64, 64)
print(code1)

code1 = torch.round(code1).reshape(64*64).cpu().detach().numpy()
code2 = np.zeros(256*16, dtype=int);
for i in range(256*16):
  code2[i] = int(code1[i])
img_en, stream_length, _, _ = dip.huffman_encode(code2)
print(img_en)
print(stream_length)

torch.Size([1, 4, 16, 16])


RuntimeError: ignored

# 新段落